In [1]:
# Import Python Libraries
import pandas as pd
import numpy as np
import datetime 
# plotting
import matplotlib as mpl
# machline learning package 
import sklearn as skl
# package to use regression
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
#Read CSV File
df = pd.read_csv("18zpallnoagi.csv")
df.head()

,STATEFIPS,STATE,ZIPCODE,AGI_STUB,N1,MARS1,MARS2,MARS4,ELF,CPREP,...,N85300,A85300,N11901,A11901,N11900,A11900,N11902,A11902,N12000,A12000
0,1,AL,0,0,2036290.0,853350.0,746450.0,393790.0,1851240.0,93480.0,...,36360.0,140470.0,388540.0,1734772.0,1602800.0,5324632.0,1570180.0,4701007.0,36520.0,559783.0
1,1,AL,35004,0,5200.0,2150.0,2100.0,820.0,4730.0,260.0,...,20.0,63.0,1000.0,3171.0,4090.0,11131.0,4050.0,10654.0,50.0,433.0
2,1,AL,35005,0,3190.0,1410.0,840.0,890.0,2880.0,160.0,...,0.0,0.0,530.0,1179.0,2630.0,7589.0,2630.0,7558.0,0.0,0.0
3,1,AL,35006,0,1240.0,490.0,590.0,140.0,1120.0,40.0,...,0.0,0.0,200.0,498.0,1010.0,2834.0,1010.0,2793.0,0.0,0.0
4,1,AL,35007,0,12050.0,4840.0,5180.0,1740.0,10580.0,840.0,...,100.0,307.0,2780.0,11619.0,9020.0,24590.0,8900.0,23889.0,150.0,668.0


In [41]:
#Remove the Aggregated Rows from the dataset
new_df = df.query("ZIPCODE!=0")
new_df = new_df.reset_index(drop=True)
new_df.shape

(27607, 153)

In [42]:
#Drop the Non-Predictors Columns from the dataset
new_df=new_df.drop(columns=['STATEFIPS','STATE','ZIPCODE','AGI_STUB','MARS1','MARS2','MARS4','ELF','CPREP','PREP','DIR_DEP','TOTAL_VITA','VITA','TCE','VITA_EIC','RAC','ELDERLY'])
new_df.head()

,N1,N2,NUMDEP,A00100,N02650,A02650,N00200,A00200,N00300,A00300,...,N85300,A85300,N11901,A11901,N11900,A11900,N11902,A11902,N12000,A12000
0,5200.0,10440.0,3420.0,302269.0,5200.0,304546.0,4480.0,245939.0,1270.0,953.0,...,20.0,63.0,1000.0,3171.0,4090.0,11131.0,4050.0,10654.0,50.0,433.0
1,3190.0,6060.0,2220.0,130555.0,3190.0,131420.0,2740.0,107771.0,430.0,361.0,...,0.0,0.0,530.0,1179.0,2630.0,7589.0,2630.0,7558.0,0.0,0.0
2,1240.0,2550.0,790.0,65411.0,1240.0,65797.0,1020.0,50659.0,260.0,115.0,...,0.0,0.0,200.0,498.0,1010.0,2834.0,1010.0,2793.0,0.0,0.0
3,12050.0,24970.0,8740.0,743431.0,12050.0,750814.0,10210.0,578197.0,2880.0,2512.0,...,100.0,307.0,2780.0,11619.0,9020.0,24590.0,8900.0,23889.0,150.0,668.0
4,7840.0,15810.0,5710.0,410413.0,7840.0,414405.0,6630.0,277183.0,1770.0,3596.0,...,110.0,498.0,1300.0,5018.0,6370.0,21697.0,6210.0,19500.0,180.0,2176.0


In [43]:
#Remove all the Number Columns as we are going to generate avarage amount per return. Use Amounts only for Predictors
for column in new_df.columns:
    if column not in ('N1','N2','NUMDEP','N03210'):
        if 'N' in column:
            new_df=new_df.drop(columns=column)
new_df.head()

,N1,N2,NUMDEP,A00100,A02650,A00200,A00300,A00600,A00650,A00700,...,A10960,A11560,A06500,A10300,A85530,A85300,A11901,A11900,A11902,A12000
0,5200.0,10440.0,3420.0,302269.0,304546.0,245939.0,953.0,1404.0,993.0,820.0,...,215.0,60.0,24437.0,26082.0,19.0,63.0,3171.0,11131.0,10654.0,433.0
1,3190.0,6060.0,2220.0,130555.0,131420.0,107771.0,361.0,277.0,189.0,310.0,...,202.0,27.0,8313.0,8913.0,0.0,0.0,1179.0,7589.0,7558.0,0.0
2,1240.0,2550.0,790.0,65411.0,65797.0,50659.0,115.0,160.0,75.0,100.0,...,56.0,12.0,4964.0,5295.0,0.0,0.0,498.0,2834.0,2793.0,0.0
3,12050.0,24970.0,8740.0,743431.0,750814.0,578197.0,2512.0,4998.0,3648.0,2183.0,...,752.0,161.0,65944.0,71665.0,96.0,307.0,11619.0,24590.0,23889.0,668.0
4,7840.0,15810.0,5710.0,410413.0,414405.0,277183.0,3596.0,9509.0,7543.0,820.0,...,347.0,89.0,37216.0,40869.0,159.0,498.0,5018.0,21697.0,19500.0,2176.0


In [44]:
#Create the Target Column
new_df['A03210_per_return']=new_df['A03210']/new_df['N03210']
new_df=new_df.fillna(0)
new_df.head()

,N1,N2,NUMDEP,A00100,A02650,A00200,A00300,A00600,A00650,A00700,...,A11560,A06500,A10300,A85530,A85300,A11901,A11900,A11902,A12000,A03210_per_return
0,5200.0,10440.0,3420.0,302269.0,304546.0,245939.0,953.0,1404.0,993.0,820.0,...,60.0,24437.0,26082.0,19.0,63.0,3171.0,11131.0,10654.0,433.0,1.067241
1,3190.0,6060.0,2220.0,130555.0,131420.0,107771.0,361.0,277.0,189.0,310.0,...,27.0,8313.0,8913.0,0.0,0.0,1179.0,7589.0,7558.0,0.0,1.125000
2,1240.0,2550.0,790.0,65411.0,65797.0,50659.0,115.0,160.0,75.0,100.0,...,12.0,4964.0,5295.0,0.0,0.0,498.0,2834.0,2793.0,0.0,0.740000
3,12050.0,24970.0,8740.0,743431.0,750814.0,578197.0,2512.0,4998.0,3648.0,2183.0,...,161.0,65944.0,71665.0,96.0,307.0,11619.0,24590.0,23889.0,668.0,1.112195
4,7840.0,15810.0,5710.0,410413.0,414405.0,277183.0,3596.0,9509.0,7543.0,820.0,...,89.0,37216.0,40869.0,159.0,498.0,5018.0,21697.0,19500.0,2176.0,1.000000


In [45]:
predictors = []
for column in new_df.columns:
    if '03210' not in column:
            predictors.append(column)

target = 'A03210_per_return' 

In [47]:
#Linear Regression Model
#Model
from sklearn.model_selection import train_test_split

x = new_df[predictors].values
y = new_df[target].values

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state=0)

linear_model = sm.OLS(y_train,X_train)
result = linear_model.fit()
print(result.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.577
Model:                            OLS   Adj. R-squared (uncentered):              0.576
Method:                 Least Squares   F-statistic:                              381.0
Date:                Wed, 07 Oct 2020   Prob (F-statistic):                        0.00
Time:                        06:52:10   Log-Likelihood:                         -17859.
No. Observations:               19324   AIC:                                  3.586e+04
Df Residuals:                   19255   BIC:                                  3.640e+04
Df Model:                          69                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [48]:
#xgBoost Regressor ML Model
import xgboost as xgb
from sklearn.metrics import mean_squared_error,r2_score

xg_reg = xgb.XGBRegressor()
xg_reg.fit(X_train,y_train)
y_test_preds = xg_reg.predict(X_test)
y_test_predict = result.predict(X_test)

print('R2_Score')
print('  OLS Model: ',r2_score(y_test,y_test_predict))
print('  RGB Model: ',r2_score(y_test,y_test_preds))

print('MSE_Score')
print('  OLS Model: ',mcean_squared_error(y_test,y_test_predict))
print('  RGB Model: ',mean_squared_error(y_test,y_test_preds))

R2_Score
  OLS Model:  -1.6702116648740715
  RGB Model:  0.749092761395866
MSE_Score
  OLS Model:  0.43230963878934786
  RGB Model:  0.04062210465090654
